In [1]:
#This cell imports the required pyspark module
import sys
!pip install pyspark 
from pyspark import SparkContext, SparkConf

sc = SparkContext("local","Project1")

In [2]:
#This should be used only when running this on google colab
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
#import the input file. I have just considered just a small portion of the file as system has very less resources. You can use the main input file path
file1 = '/content/gdrive/My Drive/CS5683/browsing_test.txt'
fileRDD = sc.textFile(file1)

In [4]:
#This shows the first 10 of the input
fileRDD.take(10)

['FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 ',
 'GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 ',
 'ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 ',
 'ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 ',
 'ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 ',
 'ELE17451 GRO73461 DAI22896 SNA99873 FRO18919 DAI50921 SNA80192 GRO75578 ',
 'ELE17451 ELE59935 FRO18919 ELE23393 SNA80192 SNA85662 SNA91554 DAI22177 ',
 'ELE17451 SNA69641 FRO18919 SNA90258 ELE28573 ELE11375 DAI14125 FRO78087 ',
 'ELE17451 GRO73461 DAI22896 SNA80192 SNA85662 SNA90258 DAI46755 FRO81176 ELE66810 DAI49199 DAI91535 GRO94758 ELE94711 DAI22177 ',
 'ELE17451 SNA69641 DAI91535 GRO94758 GRO99222 FRO76833 FRO81176 SNA80192 DAI54690 ELE37798 GRO56989 ']

In [5]:
#This cell calculates the singletons and filters them based on the required support
from operator import add
minSupport = 85#I have given 2 as the support for my small dataset to just try.
items = fileRDD.map(lambda line:line.split(" "))
itemCount = items.flatMap(lambda x: x).map(lambda item:(item, 1)).reduceByKey(add)
l1 = itemCount.filter(lambda a: a[1] > minSupport).filter(lambda a: a[0] != '')

In [6]:
#This cell shows the top 5 singletons based on the support
l1.sortBy(lambda a: a[1], ascending = False).take(5) 

[('ELE17451', 15),
 ('GRO99222', 7),
 ('SNA90258', 6),
 ('DAI22896', 6),
 ('FRO86643', 6)]

In [7]:
#This cell shows the count of singletons formed
l1.count()

20

In [8]:
#This cell gives the main definition of calculating the frequent itemsets. This method of the freq_itemsets is called in the below cell
#you can just remove the comments to view the intermediate results from each step. 
def freq_itemsets(prev, frequent_items, s, k):
  def cand_itemsets(x):
    list1 = []
    for c in frequent_items:
      if set(c).issubset(set(x[0])) == False:
        list1.append(x[0]+c)
    return(list1)
  prev1 = prev.flatMap(cand_itemsets).map(lambda x: (x,0))
  #print(prev1.take(5))
  #print(prev1.count())
  prev2 = prev1.cartesian(items)
  #print(prev2.take(5))
  #print(prev2.count())
  def frequency(y):
    if set(y[0][0]).issubset(y[1]):
    #  y[0][1] = 1
      return((y[0][0],1))
  prev3 = prev2.map(frequency).filter(lambda x: x is not None)
  #print(prev3.take(5))
  #print(prev3.count())
  prev4 = prev3.map(lambda x: (tuple(x[0]), x[1])).reduceByKey(add).filter(lambda x: x[1] > s).map(lambda x: (list(x[0]),x[1]))#instead of frozenset 'tuple' can be used
  #print(prev4.sortBy(lambda a: a[1],ascending=False).take(5))
  #print(prev4.count())
  return(prev4)

In [16]:
#The l_f is obtained by slightly changing the singletons. The for loop logic is to calculate the frequent itemsets using the above defined method of freq_itemsets() and stores them into an frequent_items RDD
#The range can be increased to increase the size of frequent itemset. If we want the itemsets till triples we need to put range(2,4), if we need till quadraples we need to put range(2,5) and so on.
l_f = l1.map(lambda x: ([x[0]],x[1]))
for k in range(2, 5):
  if k==2:
    frequent_items = freq_itemsets(l_f,l_f.map(lambda x: x[0]).collect(),minSupport, 2)
  else:
    frequent_items1 = frequent_items.filter(lambda x: len(x[0]) == k-1)
    frequent_items = frequent_items.union(freq_itemsets(frequent_items1,l_f.map(lambda x: x[0]).collect(),minSupport, 2))

In [17]:
#This contains all the singletons, pairs, triplets, quadraples
all_sets = l_f.union(frequent_items)

In [18]:
#This shows the count of all the singletons, pairs, triplets, quadraples
all_sets.count()

158

In [21]:
#This shows the top 10 of all the singletons, pairs, triplets, quadraples
all_sets.sortBy(lambda a: a[1], ascending = False).take(10) 

[(['ELE17451'], 15),
 (['GRO99222'], 7),
 (['SNA90258'], 6),
 (['DAI22896'], 6),
 (['FRO86643'], 6),
 (['FRO18919'], 6),
 (['ELE17451', 'GRO99222'], 6),
 (['GRO99222', 'ELE17451'], 6),
 (['SNA80192'], 5),
 (['GRO73461'], 5)]

In [22]:
#This cell eliminates any duplicates present in the all_sets RDD.
all_set1 = all_sets.map(lambda x: (frozenset(x[0]), x[1])).reduceByKey(min).map(lambda x: (list(x[0]),x[1]))#.collect()

In [23]:
all_set1.count()

60

In [24]:
all_set1.sortBy(lambda a: a[1], ascending = False).take(10) 

[(['ELE17451'], 15),
 (['GRO99222'], 7),
 (['DAI22896'], 6),
 (['GRO99222', 'ELE17451'], 6),
 (['SNA90258'], 6),
 (['FRO18919'], 6),
 (['FRO86643'], 6),
 (['DAI22896', 'ELE17451'], 5),
 (['FRO86643', 'ELE17451'], 5),
 (['DAI22896', 'GRO73461'], 5)]

In [25]:
#This cell creates the association rules with confidence greater than the 0.9 using the above all_sets1 RDD
c= 0.9
def confidence(x):
    if set(x[0][0]).issubset(set(x[1][0])):
      return(((x[0][0],list(set(x[1][0])-set(x[0][0]))),(x[1][1],x[1][1]/x[0][1])))
for k in range(2, 4):
  if k==2:
    frequent_items1 = all_set1.filter(lambda x: len(x[0]) == k-1).cartesian(all_set1.filter(lambda x: len(x[0]) == k)).map(confidence).filter(lambda x: x is not None).filter(lambda x: x[1][1] > c)
  else:
    #frequent_items1 = frequent_items.filter(lambda x: len(x[0]) == k-1)
    frequent_items1 = frequent_items1.union(all_set1.filter(lambda x: len(x[0]) == k-1).cartesian(all_set1.filter(lambda x: len(x[0]) == k)).map(confidence).filter(lambda x: x is not None).filter(lambda x: x[1][1] > c))

In [ ]:
#This cell displays all the association rules in the format of ([LHS], [RHS]),(support, confidence) where the LHS is left hand side of association rule and RHS is right hand side of association rules.
frequent_items1.sortBy(lambda a: a[1][1], ascending = False).collect()

In [28]:
#This cell is finally used to print the results as given in the assignment.
def printing_rules(x):
  return((str(x[0][0]) + '-->' + str(x[0][1]) + '; Confidence = '+str(x[1][1])))
frequent_items1.sortBy(lambda a: a[1][1], ascending = False).map(printing_rules).collect()

["['ELE26917']-->['GRO99222']; Confidence = 1.0",
 "['GRO39357']-->['ELE17451']; Confidence = 1.0",
 "['GRO39357']-->['FRO78087']; Confidence = 1.0",
 "['ELE28573']-->['FRO78087']; Confidence = 1.0",
 "['ELE28573']-->['ELE11375']; Confidence = 1.0",
 "['GRO73461']-->['DAI22896']; Confidence = 1.0",
 "['SNA69641']-->['ELE17451']; Confidence = 1.0",
 "['SNA99873']-->['GRO73461']; Confidence = 1.0",
 "['SNA99873']-->['DAI22896']; Confidence = 1.0",
 "['GRO56989']-->['ELE37798']; Confidence = 1.0",
 "['GRO73461']-->['ELE17451']; Confidence = 1.0",
 "['SNA99873']-->['ELE17451']; Confidence = 1.0",
 "['GRO56989']-->['ELE17451']; Confidence = 1.0",
 "['ELE37798']-->['ELE17451']; Confidence = 1.0",
 "['SNA11465']-->['ELE17451']; Confidence = 1.0",
 "['ELE37798']-->['GRO56989']; Confidence = 1.0",
 "['SNA80192']-->['ELE17451']; Confidence = 1.0",
 "['ELE11375']-->['FRO78087']; Confidence = 1.0",
 "['ELE11375']-->['ELE28573']; Confidence = 1.0",
 "['DAI22896', 'ELE17451']-->['GRO73461']; Confide

I have tried my best to not collect anything except the singletons till the last. If you couldn't run it on the large dataset, please input some small dataset. This algorithm runs perfect 
 
Thank you